# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [1]:
#%pip install duckdb
#%pip install jupysql
#%pip install duckdb-engine

In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

In [3]:
dataset_path = '/home/augusto/Desktop/dmeyf2025/Data/'#'/content/drive/MyDrive/DMEyF/2025/datos/'
dataset_file = 'competencia_01.csv'

In [4]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


In [5]:
%%sql
select
    *
from competencia_01

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria,target,targetcluster
0,249221323,202101,1,0,0,46,93,3070.24,15691.10,1278.11,...,-20833.47,0.0,2814,6743.38,6,0,9559.95,CONTINUA,0,0
1,249227600,202101,1,0,0,42,278,496.21,15321.80,418.97,...,-9662.51,0.0,5541,9419.90,8,0,1900.26,CONTINUA,0,0
2,249234235,202101,1,0,0,54,17,806.56,10549.80,174.26,...,-4692.00,0.0,577,2821.32,2,0,6885.51,CONTINUA,0,0
3,249244449,202101,1,0,0,61,290,27485.71,65621.74,23590.68,...,-20349.22,0.0,7916,23509.71,8,0,1219.92,CONTINUA,0,0
4,249244739,202101,1,0,0,66,320,3306.82,48787.99,3274.95,...,-88178.37,0.0,1958,20397.54,8,0,2627.52,CONTINUA,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,1598368433,202106,1,0,0,58,1,4123.09,4123.09,2242.51,...,0.00,0.0,28,12521.76,2,0,1747.77,None,0,0
978435,1598419415,202106,1,0,0,30,1,-2.26,-2.26,0.00,...,NaN,NaN,9,NaN,<NA>,<NA>,0.00,None,0,0
978436,1598425905,202106,1,0,0,28,1,5.92,5.92,0.00,...,NaN,NaN,20,NaN,<NA>,<NA>,0.00,None,0,0
978437,1598470389,202106,1,0,0,78,1,244.96,244.96,0.00,...,NaN,NaN,15,NaN,<NA>,<NA>,0.00,None,0,0


In [6]:
%%sql
-- Creamos una nueva tabla para no modificar la original.
-- La cláusula 'OR REPLACE' permite ejecutar esta celda varias veces sin error.
CREATE OR REPLACE TABLE competencia_02_con_features AS
SELECT
    *, -- Incluimos todas las columnas originales

    ----------------------------------------------------
    -- 1. Ratios de Rentabilidad y Valor del Cliente
    ----------------------------------------------------
    CASE WHEN cproductos > 0 THEN mrentabilidad / cproductos ELSE 0 END AS rentabilidad_por_producto,
    CASE WHEN cliente_antiguedad > 0 THEN mrentabilidad_annual / cliente_antiguedad ELSE 0 END AS rentabilidad_por_antiguedad,
    CASE WHEN mrentabilidad <> 0 THEN mcomisiones / mrentabilidad ELSE 0 END AS proporcion_comisiones_rentabilidad,
    CASE WHEN mrentabilidad <> 0 THEN mpasivos_margen / mrentabilidad ELSE 0 END AS proporcion_margen_pasivo_rentabilidad,
    CASE WHEN mrentabilidad <> 0 THEN mactivos_margen / mrentabilidad ELSE 0 END AS proporcion_margen_activo_rentabilidad,

    ----------------------------------------------------
    -- 2. Patrones de Uso de Tarjetas de Crédito
    ----------------------------------------------------
    COALESCE(mtarjeta_visa_consumo, 0) + COALESCE(mtarjeta_master_consumo, 0) AS consumo_total_tc,
    COALESCE(Visa_msaldototal, 0) + COALESCE(Master_msaldototal, 0) AS deuda_total_tc,
    COALESCE(Visa_mlimitecompra, 0) + COALESCE(Master_mlimitecompra, 0) AS limite_total_tc,
    -- Ratio de utilización del límite (una variable muy potente)
    CASE
        WHEN (COALESCE(Visa_mlimitecompra, 0) + COALESCE(Master_mlimitecompra, 0)) > 0
        THEN (COALESCE(Visa_msaldototal, 0) + COALESCE(Master_msaldototal, 0)) / (COALESCE(Visa_mlimitecompra, 0) + COALESCE(Master_mlimitecompra, 0))
        ELSE 0
    END AS ratio_utilizacion_limite_tc,
    -- Ratio de pago sobre la deuda (indica si paga el total o el mínimo)
    CASE
        WHEN (COALESCE(Visa_msaldototal, 0) + COALESCE(Master_msaldototal, 0)) > 0
        THEN (COALESCE(Visa_mpagado, 0) + COALESCE(Master_mpagado, 0)) / (COALESCE(Visa_msaldototal, 0) + COALESCE(Master_msaldototal, 0))
        ELSE 0
    END AS ratio_pago_deuda_tc,
    -- Flag de morosidad unificado
    CASE WHEN COALESCE(Master_delinquency, 0) + COALESCE(Visa_delinquency, 0) > 0 THEN 1 ELSE 0 END AS es_moroso,

    ----------------------------------------------------
    -- 3. Comportamiento Transaccional y de Cuentas
    ----------------------------------------------------
    CASE WHEN mcuentas_saldo <> 0 THEN COALESCE(mcaja_ahorro_dolares, 0) / mcuentas_saldo ELSE 0 END AS ratio_ahorro_dolares,
    CASE WHEN ctarjeta_debito_transacciones > 0 THEN mautoservicio / ctarjeta_debito_transacciones ELSE 0 END AS monto_promedio_trx_debito,
    COALESCE(mtransferencias_recibidas, 0) - COALESCE(mtransferencias_emitidas, 0) AS balance_transferencias,
    CASE WHEN mtransferencias_recibidas > 0 THEN COALESCE(mtransferencias_emitidas, 0) / mtransferencias_recibidas ELSE 0 END AS ratio_salida_entrada_transferencias,
    CASE WHEN mtransferencias_recibidas > 0 THEN COALESCE(mpayroll, 0) / mtransferencias_recibidas ELSE 0 END AS dependencia_payroll,

    ----------------------------------------------------
    -- 4. Adopción Digital y de Canales
    ----------------------------------------------------
    COALESCE(internet, 0) + COALESCE(thomebanking, 0) + COALESCE(tmobile_app, 0) AS score_adopcion_digital,
    CASE
        WHEN (COALESCE(chomebanking_transacciones, 0) + COALESCE(cmobile_app_trx, 0) + COALESCE(ccajas_transacciones, 0) + COALESCE(catm_trx, 0)) > 0
        THEN (COALESCE(chomebanking_transacciones, 0) + COALESCE(cmobile_app_trx, 0)) / (COALESCE(chomebanking_transacciones, 0) + COALESCE(cmobile_app_trx, 0) + COALESCE(ccajas_transacciones, 0) + COALESCE(catm_trx, 0))
        ELSE 0
    END AS proporcion_trx_digitales,

    ----------------------------------------------------
    -- 5. Indicadores de Riesgo y Estabilidad Financiera
    ----------------------------------------------------
    CASE
        WHEN mpayroll > 0
        THEN (COALESCE(mprestamos_personales, 0) + COALESCE(mprestamos_prendarios, 0) + COALESCE(mprestamos_hipotecarios, 0) + COALESCE(Visa_msaldototal, 0) + COALESCE(Master_msaldototal, 0)) / mpayroll
        ELSE 0
    END AS ratio_deuda_total_ingreso,
    CASE WHEN mpayroll > 0 THEN COALESCE(mcuentas_saldo, 0) / mpayroll ELSE 0 END AS ratio_saldo_cuenta_ingreso,
    CASE WHEN ccheques_emitidos > 0 THEN COALESCE(ccheques_emitidos_rechazados, 0) / ccheques_emitidos ELSE 0 END AS ratio_cheques_rechazados_emitidos,
    CASE WHEN mpayroll > 0 THEN (COALESCE(mcuenta_debitos_automaticos, 0) + COALESCE(mpagodeservicios, 0)) / mpayroll ELSE 0 END as ratio_gastos_esenciales

FROM
    competencia_01; -- Tu tabla original

,Success


In [7]:
%%sql
CREATE OR REPLACE TABLE competencia_03_con_lags_masivos AS
SELECT
    *, -- Mantenemos todas las columnas originales y los ratios

    ----------------------------------------------------
    -- Lags para Rentabilidad y Márgenes
    ----------------------------------------------------
    LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mrentabilidad_lag1,
    LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mrentabilidad_lag2,
    LAG(mrentabilidad, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mrentabilidad_lag3,
    LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mrentabilidad_annual_lag1,
    LAG(mrentabilidad_annual, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mrentabilidad_annual_lag2,
    LAG(mrentabilidad_annual, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mrentabilidad_annual_lag3,
    LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_lag1,
    LAG(mcomisiones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_lag2,
    LAG(mcomisiones, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_lag3,
    LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mactivos_margen_lag1,
    LAG(mactivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mactivos_margen_lag2,
    LAG(mactivos_margen, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mactivos_margen_lag3,
    LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpasivos_margen_lag1,
    LAG(mpasivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpasivos_margen_lag2,
    LAG(mpasivos_margen, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpasivos_margen_lag3,

    ----------------------------------------------------
    -- Lags para Saldos en Cuentas
    ----------------------------------------------------
    LAG(mcuenta_corriente_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_corriente_adicional_lag1,
    LAG(mcuenta_corriente_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_corriente_adicional_lag2,
    LAG(mcuenta_corriente_adicional, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_corriente_adicional_lag3,
    LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_corriente_lag1,
    LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_corriente_lag2,
    LAG(mcuenta_corriente, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_corriente_lag3,
    LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_lag1,
    LAG(mcaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_lag2,
    LAG(mcaja_ahorro, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_lag3,
    LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_adicional_lag1,
    LAG(mcaja_ahorro_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_adicional_lag2,
    LAG(mcaja_ahorro_adicional, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_adicional_lag3,
    LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_dolares_lag1,
    LAG(mcaja_ahorro_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_dolares_lag2,
    LAG(mcaja_ahorro_dolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcaja_ahorro_dolares_lag3,
    LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuentas_saldo_lag1,
    LAG(mcuentas_saldo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuentas_saldo_lag2,
    LAG(mcuentas_saldo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuentas_saldo_lag3,

    ----------------------------------------------------
    -- Lags para Consumos y Transacciones
    ----------------------------------------------------
    LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mautoservicio_lag1,
    LAG(mautoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mautoservicio_lag2,
    LAG(mautoservicio, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mautoservicio_lag3,
    LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_visa_consumo_lag1,
    LAG(mtarjeta_visa_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_visa_consumo_lag2,
    LAG(mtarjeta_visa_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_visa_consumo_lag3,
    LAG(mtarjeta_master_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_master_consumo_lag1,
    LAG(mtarjeta_master_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_master_consumo_lag2,
    LAG(mtarjeta_master_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_master_consumo_lag3,

    ----------------------------------------------------
    -- Lags para Deudas por Préstamos
    ----------------------------------------------------
    LAG(mprestamos_personales, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_personales_lag1,
    LAG(mprestamos_personales, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_personales_lag2,
    LAG(mprestamos_personales, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_personales_lag3,
    LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_prendarios_lag1,
    LAG(mprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_prendarios_lag2,
    LAG(mprestamos_prendarios, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_prendarios_lag3,
    LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_hipotecarios_lag1,
    LAG(mprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_hipotecarios_lag2,
    LAG(mprestamos_hipotecarios, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mprestamos_hipotecarios_lag3,

    ----------------------------------------------------
    -- Lags para Inversiones
    ----------------------------------------------------
    LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mplazo_fijo_dolares_lag1,
    LAG(mplazo_fijo_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mplazo_fijo_dolares_lag2,
    LAG(mplazo_fijo_dolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mplazo_fijo_dolares_lag3,
    LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mplazo_fijo_pesos_lag1,
    LAG(mplazo_fijo_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mplazo_fijo_pesos_lag2,
    LAG(mplazo_fijo_pesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mplazo_fijo_pesos_lag3,
    LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion1_pesos_lag1,
    LAG(minversion1_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion1_pesos_lag2,
    LAG(minversion1_pesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion1_pesos_lag3,
    LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion1_dolares_lag1,
    LAG(minversion1_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion1_dolares_lag2,
    LAG(minversion1_dolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion1_dolares_lag3,
    LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion2_lag1,
    LAG(minversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion2_lag2,
    LAG(minversion2, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as minversion2_lag3,

    ----------------------------------------------------
    -- Lags para Ingresos y Pagos
    ----------------------------------------------------
    LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpayroll_lag1,
    LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpayroll_lag2,
    LAG(mpayroll, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpayroll_lag3,
    LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpayroll2_lag1,
    LAG(mpayroll2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpayroll2_lag2,
    LAG(mpayroll2, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpayroll2_lag3,
    LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_debitos_automaticos_lag1,
    LAG(mcuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_debitos_automaticos_lag2,
    LAG(mcuenta_debitos_automaticos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcuenta_debitos_automaticos_lag3,
    LAG(mttarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mttarjeta_visa_debitos_automaticos_lag1,
    LAG(mttarjeta_visa_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mttarjeta_visa_debitos_automaticos_lag2,
    LAG(mttarjeta_visa_debitos_automaticos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mttarjeta_visa_debitos_automaticos_lag3,
    LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mttarjeta_master_debitos_automaticos_lag1,
    LAG(mttarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mttarjeta_master_debitos_automaticos_lag2,
    LAG(mttarjeta_master_debitos_automaticos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mttarjeta_master_debitos_automaticos_lag3,
    LAG(mpagodeservicios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpagodeservicios_lag1,
    LAG(mpagodeservicios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpagodeservicios_lag2,
    LAG(mpagodeservicios, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpagodeservicios_lag3,
    LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpagomiscuentas_lag1,
    LAG(mpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpagomiscuentas_lag2,
    LAG(mpagomiscuentas, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mpagomiscuentas_lag3,

    ----------------------------------------------------
    -- Lags para Descuentos y Comisiones
    ----------------------------------------------------
    LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcajeros_propios_descuentos_lag1,
    LAG(mcajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcajeros_propios_descuentos_lag2,
    LAG(mcajeros_propios_descuentos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcajeros_propios_descuentos_lag3,
    LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_visa_descuentos_lag1,
    LAG(mtarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_visa_descuentos_lag2,
    LAG(mtarjeta_visa_descuentos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_visa_descuentos_lag3,
    LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_master_descuentos_lag1,
    LAG(mtarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_master_descuentos_lag2,
    LAG(mtarjeta_master_descuentos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtarjeta_master_descuentos_lag3,
    LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_mantenimiento_lag1,
    LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_mantenimiento_lag2,
    LAG(mcomisiones_mantenimiento, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_mantenimiento_lag3,
    LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_otras_lag1,
    LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_otras_lag2,
    LAG(mcomisiones_otras, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcomisiones_otras_lag3,

    ----------------------------------------------------
    -- Lags para Forex y Transferencias
    ----------------------------------------------------
    LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mforex_buy_lag1,
    LAG(mforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mforex_buy_lag2,
    LAG(mforex_buy, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mforex_buy_lag3,
    LAG(mforex_sell, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mforex_sell_lag1,
    LAG(mforex_sell, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mforex_sell_lag2,
    LAG(mforex_sell, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mforex_sell_lag3,
    LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtransferencias_recibidas_lag1,
    LAG(mtransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtransferencias_recibidas_lag2,
    LAG(mtransferencias_recibidas, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtransferencias_recibidas_lag3,
    LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtransferencias_emitidas_lag1,
    LAG(mtransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtransferencias_emitidas_lag2,
    LAG(mtransferencias_emitidas, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mtransferencias_emitidas_lag3,

    ----------------------------------------------------
    -- Lags para Cheques y ATM
    ----------------------------------------------------
    LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mextraccion_autoservicio_lag1,
    LAG(mextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mextraccion_autoservicio_lag2,
    LAG(mextraccion_autoservicio, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mextraccion_autoservicio_lag3,
    LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_depositados_lag1,
    LAG(mcheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_depositados_lag2,
    LAG(mcheques_depositados, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_depositados_lag3,
    LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_emitidos_lag1,
    LAG(mcheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_emitidos_lag2,
    LAG(mcheques_emitidos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_emitidos_lag3,
    LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_depositados_rechazados_lag1,
    LAG(mcheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_depositados_rechazados_lag2,
    LAG(mcheques_depositados_rechazados, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_depositados_rechazados_lag3,
    LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_emitidos_rechazados_lag1,
    LAG(mcheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_emitidos_rechazados_lag2,
    LAG(mcheques_emitidos_rechazados, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as mcheques_emitidos_rechazados_lag3,
    LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as matm_lag1,
    LAG(matm, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as matm_lag2,
    LAG(matm, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as matm_lag3,
    LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as matm_other_lag1,
    LAG(matm_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as matm_other_lag2,
    LAG(matm_other, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as matm_other_lag3,

    ----------------------------------------------------
    -- Lags para Tarjeta Master
    ----------------------------------------------------
    LAG(Master_mfinanciacion_limite, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mfinanciacion_limite_lag1,
    LAG(Master_mfinanciacion_limite, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mfinanciacion_limite_lag2,
    LAG(Master_mfinanciacion_limite, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mfinanciacion_limite_lag3,
    LAG(Master_msaldototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldototal_lag1,
    LAG(Master_msaldototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldototal_lag2,
    LAG(Master_msaldototal, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldototal_lag3,
    LAG(Master_msaldopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldopesos_lag1,
    LAG(Master_msaldopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldopesos_lag2,
    LAG(Master_msaldopesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldopesos_lag3,
    LAG(Master_msaldodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldodolares_lag1,
    LAG(Master_msaldodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldodolares_lag2,
    LAG(Master_msaldodolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_msaldodolares_lag3,
    LAG(Master_mconsumospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumospesos_lag1,
    LAG(Master_mconsumospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumospesos_lag2,
    LAG(Master_mconsumospesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumospesos_lag3,
    LAG(Master_mconsumosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumosdolares_lag1,
    LAG(Master_mconsumosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumosdolares_lag2,
    LAG(Master_mconsumosdolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumosdolares_lag3,
    LAG(Master_mlimitecompra, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mlimitecompra_lag1,
    LAG(Master_mlimitecompra, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mlimitecompra_lag2,
    LAG(Master_mlimitecompra, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mlimitecompra_lag3,
    LAG(Master_madelantopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_madelantopesos_lag1,
    LAG(Master_madelantopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_madelantopesos_lag2,
    LAG(Master_madelantopesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_madelantopesos_lag3,
    LAG(Master_madelantodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_madelantodolares_lag1,
    LAG(Master_madelantodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_madelantodolares_lag2,
    LAG(Master_madelantodolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_madelantodolares_lag3,
    LAG(Master_mpagado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagado_lag1,
    LAG(Master_mpagado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagado_lag2,
    LAG(Master_mpagado, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagado_lag3,
    LAG(Master_mpagospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagospesos_lag1,
    LAG(Master_mpagospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagospesos_lag2,
    LAG(Master_mpagospesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagospesos_lag3,
    LAG(Master_mpagosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagosdolares_lag1,
    LAG(Master_mpagosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagosdolares_lag2,
    LAG(Master_mpagosdolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagosdolares_lag3,
    LAG(Master_mconsumototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumototal_lag1,
    LAG(Master_mconsumototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumototal_lag2,
    LAG(Master_mconsumototal, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mconsumototal_lag3,
    LAG(Master_mpagominimo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagominimo_lag1,
    LAG(Master_mpagominimo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagominimo_lag2,
    LAG(Master_mpagominimo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Master_mpagominimo_lag3,

    ----------------------------------------------------
    -- Lags para Tarjeta Visa
    ----------------------------------------------------
    LAG(Visa_mfinanciacion_limite, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mfinanciacion_limite_lag1,
    LAG(Visa_mfinanciacion_limite, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mfinanciacion_limite_lag2,
    LAG(Visa_mfinanciacion_limite, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mfinanciacion_limite_lag3,
    LAG(Visa_msaldototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldototal_lag1,
    LAG(Visa_msaldototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldototal_lag2,
    LAG(Visa_msaldototal, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldototal_lag3,
    LAG(Visa_msaldopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldopesos_lag1,
    LAG(Visa_msaldopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldopesos_lag2,
    LAG(Visa_msaldopesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldopesos_lag3,
    LAG(Visa_msaldodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldodolares_lag1,
    LAG(Visa_msaldodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldodolares_lag2,
    LAG(Visa_msaldodolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_msaldodolares_lag3,
    LAG(Visa_mconsumospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumospesos_lag1,
    LAG(Visa_mconsumospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumospesos_lag2,
    LAG(Visa_mconsumospesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumospesos_lag3,
    LAG(Visa_mconsumosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumosdolares_lag1,
    LAG(Visa_mconsumosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumosdolares_lag2,
    LAG(Visa_mconsumosdolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumosdolares_lag3,
    LAG(Visa_mlimitecompra, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mlimitecompra_lag1,
    LAG(Visa_mlimitecompra, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mlimitecompra_lag2,
    LAG(Visa_mlimitecompra, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mlimitecompra_lag3,
    LAG(Visa_madelantopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_madelantopesos_lag1,
    LAG(Visa_madelantopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_madelantopesos_lag2,
    LAG(Visa_madelantopesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_madelantopesos_lag3,
    LAG(Visa_madelantodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_madelantodolares_lag1,
    LAG(Visa_madelantodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_madelantodolares_lag2,
    LAG(Visa_madelantodolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_madelantodolares_lag3,
    LAG(Visa_mpagado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagado_lag1,
    LAG(Visa_mpagado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagado_lag2,
    LAG(Visa_mpagado, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagado_lag3,
    LAG(Visa_mpagospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagospesos_lag1,
    LAG(Visa_mpagospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagospesos_lag2,
    LAG(Visa_mpagospesos, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagospesos_lag3,
    LAG(Visa_mpagosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagosdolares_lag1,
    LAG(Visa_mpagosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagosdolares_lag2,
    LAG(Visa_mpagosdolares, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagosdolares_lag3,
    LAG(Visa_mconsumototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumototal_lag1,
    LAG(Visa_mconsumototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumototal_lag2,
    LAG(Visa_mconsumototal, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mconsumototal_lag3,
    LAG(Visa_mpagominimo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagominimo_lag1,
    LAG(Visa_mpagominimo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagominimo_lag2,
    LAG(Visa_mpagominimo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) as Visa_mpagominimo_lag3
FROM
    competencia_02_con_features;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [8]:
%%sql
CREATE OR REPLACE TABLE competencia_04_con_deltas AS
SELECT
    *, -- Mantenemos todas las columnas originales, ratios y lags
    ----------------------------------------------------
    -- Deltas para Rentabilidad y Márgenes
    ----------------------------------------------------
    mrentabilidad - COALESCE(mrentabilidad_lag1, mrentabilidad) as mrentabilidad_delta1,
    mrentabilidad - COALESCE(mrentabilidad_lag2, mrentabilidad) as mrentabilidad_delta2,
    mrentabilidad - COALESCE(mrentabilidad_lag3, mrentabilidad) as mrentabilidad_delta3,
    mrentabilidad_annual - COALESCE(mrentabilidad_annual_lag1, mrentabilidad_annual) as mrentabilidad_annual_delta1,
    mrentabilidad_annual - COALESCE(mrentabilidad_annual_lag2, mrentabilidad_annual) as mrentabilidad_annual_delta2,
    mrentabilidad_annual - COALESCE(mrentabilidad_annual_lag3, mrentabilidad_annual) as mrentabilidad_annual_delta3,
    mcomisiones - COALESCE(mcomisiones_lag1, mcomisiones) as mcomisiones_delta1,
    mcomisiones - COALESCE(mcomisiones_lag2, mcomisiones) as mcomisiones_delta2,
    mcomisiones - COALESCE(mcomisiones_lag3, mcomisiones) as mcomisiones_delta3,
    mactivos_margen - COALESCE(mactivos_margen_lag1, mactivos_margen) as mactivos_margen_delta1,
    mactivos_margen - COALESCE(mactivos_margen_lag2, mactivos_margen) as mactivos_margen_delta2,
    mactivos_margen - COALESCE(mactivos_margen_lag3, mactivos_margen) as mactivos_margen_delta3,
    mpasivos_margen - COALESCE(mpasivos_margen_lag1, mpasivos_margen) as mpasivos_margen_delta1,
    mpasivos_margen - COALESCE(mpasivos_margen_lag2, mpasivos_margen) as mpasivos_margen_delta2,
    mpasivos_margen - COALESCE(mpasivos_margen_lag3, mpasivos_margen) as mpasivos_margen_delta3,

    ----------------------------------------------------
    -- Deltas para Saldos en Cuentas
    ----------------------------------------------------
    mcuenta_corriente_adicional - COALESCE(mcuenta_corriente_adicional_lag1, mcuenta_corriente_adicional) as mcuenta_corriente_adicional_delta1,
    mcuenta_corriente_adicional - COALESCE(mcuenta_corriente_adicional_lag2, mcuenta_corriente_adicional) as mcuenta_corriente_adicional_delta2,
    mcuenta_corriente_adicional - COALESCE(mcuenta_corriente_adicional_lag3, mcuenta_corriente_adicional) as mcuenta_corriente_adicional_delta3,
    mcuenta_corriente - COALESCE(mcuenta_corriente_lag1, mcuenta_corriente) as mcuenta_corriente_delta1,
    mcuenta_corriente - COALESCE(mcuenta_corriente_lag2, mcuenta_corriente) as mcuenta_corriente_delta2,
    mcuenta_corriente - COALESCE(mcuenta_corriente_lag3, mcuenta_corriente) as mcuenta_corriente_delta3,
    mcaja_ahorro - COALESCE(mcaja_ahorro_lag1, mcaja_ahorro) as mcaja_ahorro_delta1,
    mcaja_ahorro - COALESCE(mcaja_ahorro_lag2, mcaja_ahorro) as mcaja_ahorro_delta2,
    mcaja_ahorro - COALESCE(mcaja_ahorro_lag3, mcaja_ahorro) as mcaja_ahorro_delta3,
    mcaja_ahorro_adicional - COALESCE(mcaja_ahorro_adicional_lag1, mcaja_ahorro_adicional) as mcaja_ahorro_adicional_delta1,
    mcaja_ahorro_adicional - COALESCE(mcaja_ahorro_adicional_lag2, mcaja_ahorro_adicional) as mcaja_ahorro_adicional_delta2,
    mcaja_ahorro_adicional - COALESCE(mcaja_ahorro_adicional_lag3, mcaja_ahorro_adicional) as mcaja_ahorro_adicional_delta3,
    mcaja_ahorro_dolares - COALESCE(mcaja_ahorro_dolares_lag1, mcaja_ahorro_dolares) as mcaja_ahorro_dolares_delta1,
    mcaja_ahorro_dolares - COALESCE(mcaja_ahorro_dolares_lag2, mcaja_ahorro_dolares) as mcaja_ahorro_dolares_delta2,
    mcaja_ahorro_dolares - COALESCE(mcaja_ahorro_dolares_lag3, mcaja_ahorro_dolares) as mcaja_ahorro_dolares_delta3,
    mcuentas_saldo - COALESCE(mcuentas_saldo_lag1, mcuentas_saldo) as mcuentas_saldo_delta1,
    mcuentas_saldo - COALESCE(mcuentas_saldo_lag2, mcuentas_saldo) as mcuentas_saldo_delta2,
    mcuentas_saldo - COALESCE(mcuentas_saldo_lag3, mcuentas_saldo) as mcuentas_saldo_delta3,

    ----------------------------------------------------
    -- Deltas para Consumos y Transacciones
    ----------------------------------------------------
    mautoservicio - COALESCE(mautoservicio_lag1, mautoservicio) as mautoservicio_delta1,
    mautoservicio - COALESCE(mautoservicio_lag2, mautoservicio) as mautoservicio_delta2,
    mautoservicio - COALESCE(mautoservicio_lag3, mautoservicio) as mautoservicio_delta3,
    mtarjeta_visa_consumo - COALESCE(mtarjeta_visa_consumo_lag1, mtarjeta_visa_consumo) as mtarjeta_visa_consumo_delta1,
    mtarjeta_visa_consumo - COALESCE(mtarjeta_visa_consumo_lag2, mtarjeta_visa_consumo) as mtarjeta_visa_consumo_delta2,
    mtarjeta_visa_consumo - COALESCE(mtarjeta_visa_consumo_lag3, mtarjeta_visa_consumo) as mtarjeta_visa_consumo_delta3,
    mtarjeta_master_consumo - COALESCE(mtarjeta_master_consumo_lag1, mtarjeta_master_consumo) as mtarjeta_master_consumo_delta1,
    mtarjeta_master_consumo - COALESCE(mtarjeta_master_consumo_lag2, mtarjeta_master_consumo) as mtarjeta_master_consumo_delta2,
    mtarjeta_master_consumo - COALESCE(mtarjeta_master_consumo_lag3, mtarjeta_master_consumo) as mtarjeta_master_consumo_delta3,

    ----------------------------------------------------
    -- Deltas para Deudas por Préstamos
    ----------------------------------------------------
    mprestamos_personales - COALESCE(mprestamos_personales_lag1, mprestamos_personales) as mprestamos_personales_delta1,
    mprestamos_personales - COALESCE(mprestamos_personales_lag2, mprestamos_personales) as mprestamos_personales_delta2,
    mprestamos_personales - COALESCE(mprestamos_personales_lag3, mprestamos_personales) as mprestamos_personales_delta3,
    mprestamos_prendarios - COALESCE(mprestamos_prendarios_lag1, mprestamos_prendarios) as mprestamos_prendarios_delta1,
    mprestamos_prendarios - COALESCE(mprestamos_prendarios_lag2, mprestamos_prendarios) as mprestamos_prendarios_delta2,
    mprestamos_prendarios - COALESCE(mprestamos_prendarios_lag3, mprestamos_prendarios) as mprestamos_prendarios_delta3,
    mprestamos_hipotecarios - COALESCE(mprestamos_hipotecarios_lag1, mprestamos_hipotecarios) as mprestamos_hipotecarios_delta1,
    mprestamos_hipotecarios - COALESCE(mprestamos_hipotecarios_lag2, mprestamos_hipotecarios) as mprestamos_hipotecarios_delta2,
    mprestamos_hipotecarios - COALESCE(mprestamos_hipotecarios_lag3, mprestamos_hipotecarios) as mprestamos_hipotecarios_delta3,

    ----------------------------------------------------
    -- Deltas para Inversiones
    ----------------------------------------------------
    mplazo_fijo_dolares - COALESCE(mplazo_fijo_dolares_lag1, mplazo_fijo_dolares) as mplazo_fijo_dolares_delta1,
    mplazo_fijo_dolares - COALESCE(mplazo_fijo_dolares_lag2, mplazo_fijo_dolares) as mplazo_fijo_dolares_delta2,
    mplazo_fijo_dolares - COALESCE(mplazo_fijo_dolares_lag3, mplazo_fijo_dolares) as mplazo_fijo_dolares_delta3,
    mplazo_fijo_pesos - COALESCE(mplazo_fijo_pesos_lag1, mplazo_fijo_pesos) as mplazo_fijo_pesos_delta1,
    mplazo_fijo_pesos - COALESCE(mplazo_fijo_pesos_lag2, mplazo_fijo_pesos) as mplazo_fijo_pesos_delta2,
    mplazo_fijo_pesos - COALESCE(mplazo_fijo_pesos_lag3, mplazo_fijo_pesos) as mplazo_fijo_pesos_delta3,
    minversion1_pesos - COALESCE(minversion1_pesos_lag1, minversion1_pesos) as minversion1_pesos_delta1,
    minversion1_pesos - COALESCE(minversion1_pesos_lag2, minversion1_pesos) as minversion1_pesos_delta2,
    minversion1_pesos - COALESCE(minversion1_pesos_lag3, minversion1_pesos) as minversion1_pesos_delta3,
    minversion1_dolares - COALESCE(minversion1_dolares_lag1, minversion1_dolares) as minversion1_dolares_delta1,
    minversion1_dolares - COALESCE(minversion1_dolares_lag2, minversion1_dolares) as minversion1_dolares_delta2,
    minversion1_dolares - COALESCE(minversion1_dolares_lag3, minversion1_dolares) as minversion1_dolares_delta3,
    minversion2 - COALESCE(minversion2_lag1, minversion2) as minversion2_delta1,
    minversion2 - COALESCE(minversion2_lag2, minversion2) as minversion2_delta2,
    minversion2 - COALESCE(minversion2_lag3, minversion2) as minversion2_delta3,

    ----------------------------------------------------
    -- Deltas para Ingresos y Pagos
    ----------------------------------------------------
    mpayroll - COALESCE(mpayroll_lag1, mpayroll) as mpayroll_delta1,
    mpayroll - COALESCE(mpayroll_lag2, mpayroll) as mpayroll_delta2,
    mpayroll - COALESCE(mpayroll_lag3, mpayroll) as mpayroll_delta3,
    mpayroll2 - COALESCE(mpayroll2_lag1, mpayroll2) as mpayroll2_delta1,
    mpayroll2 - COALESCE(mpayroll2_lag2, mpayroll2) as mpayroll2_delta2,
    mpayroll2 - COALESCE(mpayroll2_lag3, mpayroll2) as mpayroll2_delta3,
    mcuenta_debitos_automaticos - COALESCE(mcuenta_debitos_automaticos_lag1, mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_delta1,
    mcuenta_debitos_automaticos - COALESCE(mcuenta_debitos_automaticos_lag2, mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_delta2,
    mcuenta_debitos_automaticos - COALESCE(mcuenta_debitos_automaticos_lag3, mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_delta3,
    mttarjeta_visa_debitos_automaticos - COALESCE(mttarjeta_visa_debitos_automaticos_lag1, mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_delta1,
    mttarjeta_visa_debitos_automaticos - COALESCE(mttarjeta_visa_debitos_automaticos_lag2, mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_delta2,
    mttarjeta_visa_debitos_automaticos - COALESCE(mttarjeta_visa_debitos_automaticos_lag3, mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_delta3,
    mttarjeta_master_debitos_automaticos - COALESCE(mttarjeta_master_debitos_automaticos_lag1, mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_delta1,
    mttarjeta_master_debitos_automaticos - COALESCE(mttarjeta_master_debitos_automaticos_lag2, mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_delta2,
    mttarjeta_master_debitos_automaticos - COALESCE(mttarjeta_master_debitos_automaticos_lag3, mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_delta3,
    mpagodeservicios - COALESCE(mpagodeservicios_lag1, mpagodeservicios) as mpagodeservicios_delta1,
    mpagodeservicios - COALESCE(mpagodeservicios_lag2, mpagodeservicios) as mpagodeservicios_delta2,
    mpagodeservicios - COALESCE(mpagodeservicios_lag3, mpagodeservicios) as mpagodeservicios_delta3,
    mpagomiscuentas - COALESCE(mpagomiscuentas_lag1, mpagomiscuentas) as mpagomiscuentas_delta1,
    mpagomiscuentas - COALESCE(mpagomiscuentas_lag2, mpagomiscuentas) as mpagomiscuentas_delta2,
    mpagomiscuentas - COALESCE(mpagomiscuentas_lag3, mpagomiscuentas) as mpagomiscuentas_delta3,

    ----------------------------------------------------
    -- Deltas para Descuentos y Comisiones
    ----------------------------------------------------
    mcajeros_propios_descuentos - COALESCE(mcajeros_propios_descuentos_lag1, mcajeros_propios_descuentos) as mcajeros_propios_descuentos_delta1,
    mcajeros_propios_descuentos - COALESCE(mcajeros_propios_descuentos_lag2, mcajeros_propios_descuentos) as mcajeros_propios_descuentos_delta2,
    mcajeros_propios_descuentos - COALESCE(mcajeros_propios_descuentos_lag3, mcajeros_propios_descuentos) as mcajeros_propios_descuentos_delta3,
    mtarjeta_visa_descuentos - COALESCE(mtarjeta_visa_descuentos_lag1, mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_delta1,
    mtarjeta_visa_descuentos - COALESCE(mtarjeta_visa_descuentos_lag2, mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_delta2,
    mtarjeta_visa_descuentos - COALESCE(mtarjeta_visa_descuentos_lag3, mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_delta3,
    mtarjeta_master_descuentos - COALESCE(mtarjeta_master_descuentos_lag1, mtarjeta_master_descuentos) as mtarjeta_master_descuentos_delta1,
    mtarjeta_master_descuentos - COALESCE(mtarjeta_master_descuentos_lag2, mtarjeta_master_descuentos) as mtarjeta_master_descuentos_delta2,
    mtarjeta_master_descuentos - COALESCE(mtarjeta_master_descuentos_lag3, mtarjeta_master_descuentos) as mtarjeta_master_descuentos_delta3,
    mcomisiones_mantenimiento - COALESCE(mcomisiones_mantenimiento_lag1, mcomisiones_mantenimiento) as mcomisiones_mantenimiento_delta1,
    mcomisiones_mantenimiento - COALESCE(mcomisiones_mantenimiento_lag2, mcomisiones_mantenimiento) as mcomisiones_mantenimiento_delta2,
    mcomisiones_mantenimiento - COALESCE(mcomisiones_mantenimiento_lag3, mcomisiones_mantenimiento) as mcomisiones_mantenimiento_delta3,
    mcomisiones_otras - COALESCE(mcomisiones_otras_lag1, mcomisiones_otras) as mcomisiones_otras_delta1,
    mcomisiones_otras - COALESCE(mcomisiones_otras_lag2, mcomisiones_otras) as mcomisiones_otras_delta2,
    mcomisiones_otras - COALESCE(mcomisiones_otras_lag3, mcomisiones_otras) as mcomisiones_otras_delta3,

    ----------------------------------------------------
    -- Deltas para Forex y Transferencias
    ----------------------------------------------------
    mforex_buy - COALESCE(mforex_buy_lag1, mforex_buy) as mforex_buy_delta1,
    mforex_buy - COALESCE(mforex_buy_lag2, mforex_buy) as mforex_buy_delta2,
    mforex_buy - COALESCE(mforex_buy_lag3, mforex_buy) as mforex_buy_delta3,
    mforex_sell - COALESCE(mforex_sell_lag1, mforex_sell) as mforex_sell_delta1,
    mforex_sell - COALESCE(mforex_sell_lag2, mforex_sell) as mforex_sell_delta2,
    mforex_sell - COALESCE(mforex_sell_lag3, mforex_sell) as mforex_sell_delta3,
    mtransferencias_recibidas - COALESCE(mtransferencias_recibidas_lag1, mtransferencias_recibidas) as mtransferencias_recibidas_delta1,
    mtransferencias_recibidas - COALESCE(mtransferencias_recibidas_lag2, mtransferencias_recibidas) as mtransferencias_recibidas_delta2,
    mtransferencias_recibidas - COALESCE(mtransferencias_recibidas_lag3, mtransferencias_recibidas) as mtransferencias_recibidas_delta3,
    mtransferencias_emitidas - COALESCE(mtransferencias_emitidas_lag1, mtransferencias_emitidas) as mtransferencias_emitidas_delta1,
    mtransferencias_emitidas - COALESCE(mtransferencias_emitidas_lag2, mtransferencias_emitidas) as mtransferencias_emitidas_delta2,
    mtransferencias_emitidas - COALESCE(mtransferencias_emitidas_lag3, mtransferencias_emitidas) as mtransferencias_emitidas_delta3,

    ----------------------------------------------------
    -- Deltas para Cheques y ATM
    ----------------------------------------------------
    mextraccion_autoservicio - COALESCE(mextraccion_autoservicio_lag1, mextraccion_autoservicio) as mextraccion_autoservicio_delta1,
    mextraccion_autoservicio - COALESCE(mextraccion_autoservicio_lag2, mextraccion_autoservicio) as mextraccion_autoservicio_delta2,
    mextraccion_autoservicio - COALESCE(mextraccion_autoservicio_lag3, mextraccion_autoservicio) as mextraccion_autoservicio_delta3,
    mcheques_depositados - COALESCE(mcheques_depositados_lag1, mcheques_depositados) as mcheques_depositados_delta1,
    mcheques_depositados - COALESCE(mcheques_depositados_lag2, mcheques_depositados) as mcheques_depositados_delta2,
    mcheques_depositados - COALESCE(mcheques_depositados_lag3, mcheques_depositados) as mcheques_depositados_delta3,
    mcheques_emitidos - COALESCE(mcheques_emitidos_lag1, mcheques_emitidos) as mcheques_emitidos_delta1,
    mcheques_emitidos - COALESCE(mcheques_emitidos_lag2, mcheques_emitidos) as mcheques_emitidos_delta2,
    mcheques_emitidos - COALESCE(mcheques_emitidos_lag3, mcheques_emitidos) as mcheques_emitidos_delta3,
    mcheques_depositados_rechazados - COALESCE(mcheques_depositados_rechazados_lag1, mcheques_depositados_rechazados) as mcheques_depositados_rechazados_delta1,
    mcheques_depositados_rechazados - COALESCE(mcheques_depositados_rechazados_lag2, mcheques_depositados_rechazados) as mcheques_depositados_rechazados_delta2,
    mcheques_depositados_rechazados - COALESCE(mcheques_depositados_rechazados_lag3, mcheques_depositados_rechazados) as mcheques_depositados_rechazados_delta3,
    mcheques_emitidos_rechazados - COALESCE(mcheques_emitidos_rechazados_lag1, mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_delta1,
    mcheques_emitidos_rechazados - COALESCE(mcheques_emitidos_rechazados_lag2, mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_delta2,
    mcheques_emitidos_rechazados - COALESCE(mcheques_emitidos_rechazados_lag3, mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_delta3,
    matm - COALESCE(matm_lag1, matm) as matm_delta1,
    matm - COALESCE(matm_lag2, matm) as matm_delta2,
    matm - COALESCE(matm_lag3, matm) as matm_delta3,
    matm_other - COALESCE(matm_other_lag1, matm_other) as matm_other_delta1,
    matm_other - COALESCE(matm_other_lag2, matm_other) as matm_other_delta2,
    matm_other - COALESCE(matm_other_lag3, matm_other) as matm_other_delta3,

    ----------------------------------------------------
    -- Deltas para Tarjeta Master
    ----------------------------------------------------
    Master_mfinanciacion_limite - COALESCE(Master_mfinanciacion_limite_lag1, Master_mfinanciacion_limite) as Master_mfinanciacion_limite_delta1,
    Master_mfinanciacion_limite - COALESCE(Master_mfinanciacion_limite_lag2, Master_mfinanciacion_limite) as Master_mfinanciacion_limite_delta2,
    Master_mfinanciacion_limite - COALESCE(Master_mfinanciacion_limite_lag3, Master_mfinanciacion_limite) as Master_mfinanciacion_limite_delta3,
    Master_msaldototal - COALESCE(Master_msaldototal_lag1, Master_msaldototal) as Master_msaldototal_delta1,
    Master_msaldototal - COALESCE(Master_msaldototal_lag2, Master_msaldototal) as Master_msaldototal_delta2,
    Master_msaldototal - COALESCE(Master_msaldototal_lag3, Master_msaldototal) as Master_msaldototal_delta3,
    Master_msaldopesos - COALESCE(Master_msaldopesos_lag1, Master_msaldopesos) as Master_msaldopesos_delta1,
    Master_msaldopesos - COALESCE(Master_msaldopesos_lag2, Master_msaldopesos) as Master_msaldopesos_delta2,
    Master_msaldopesos - COALESCE(Master_msaldopesos_lag3, Master_msaldopesos) as Master_msaldopesos_delta3,
    Master_msaldodolares - COALESCE(Master_msaldodolares_lag1, Master_msaldodolares) as Master_msaldodolares_delta1,
    Master_msaldodolares - COALESCE(Master_msaldodolares_lag2, Master_msaldodolares) as Master_msaldodolares_delta2,
    Master_msaldodolares - COALESCE(Master_msaldodolares_lag3, Master_msaldodolares) as Master_msaldodolares_delta3,
    Master_mconsumospesos - COALESCE(Master_mconsumospesos_lag1, Master_mconsumospesos) as Master_mconsumospesos_delta1,
    Master_mconsumospesos - COALESCE(Master_mconsumospesos_lag2, Master_mconsumospesos) as Master_mconsumospesos_delta2,
    Master_mconsumospesos - COALESCE(Master_mconsumospesos_lag3, Master_mconsumospesos) as Master_mconsumospesos_delta3,
    Master_mconsumosdolares - COALESCE(Master_mconsumosdolares_lag1, Master_mconsumosdolares) as Master_mconsumosdolares_delta1,
    Master_mconsumosdolares - COALESCE(Master_mconsumosdolares_lag2, Master_mconsumosdolares) as Master_mconsumosdolares_delta2,
    Master_mconsumosdolares - COALESCE(Master_mconsumosdolares_lag3, Master_mconsumosdolares) as Master_mconsumosdolares_delta3,
    Master_mlimitecompra - COALESCE(Master_mlimitecompra_lag1, Master_mlimitecompra) as Master_mlimitecompra_delta1,
    Master_mlimitecompra - COALESCE(Master_mlimitecompra_lag2, Master_mlimitecompra) as Master_mlimitecompra_delta2,
    Master_mlimitecompra - COALESCE(Master_mlimitecompra_lag3, Master_mlimitecompra) as Master_mlimitecompra_delta3,
    Master_madelantopesos - COALESCE(Master_madelantopesos_lag1, Master_madelantopesos) as Master_madelantopesos_delta1,
    Master_madelantopesos - COALESCE(Master_madelantopesos_lag2, Master_madelantopesos) as Master_madelantopesos_delta2,
    Master_madelantopesos - COALESCE(Master_madelantopesos_lag3, Master_madelantopesos) as Master_madelantopesos_delta3,
    Master_madelantodolares - COALESCE(Master_madelantodolares_lag1, Master_madelantodolares) as Master_madelantodolares_delta1,
    Master_madelantodolares - COALESCE(Master_madelantodolares_lag2, Master_madelantodolares) as Master_madelantodolares_delta2,
    Master_madelantodolares - COALESCE(Master_madelantodolares_lag3, Master_madelantodolares) as Master_madelantodolares_delta3,
    Master_mpagado - COALESCE(Master_mpagado_lag1, Master_mpagado) as Master_mpagado_delta1,
    Master_mpagado - COALESCE(Master_mpagado_lag2, Master_mpagado) as Master_mpagado_delta2,
    Master_mpagado - COALESCE(Master_mpagado_lag3, Master_mpagado) as Master_mpagado_delta3,
    Master_mpagospesos - COALESCE(Master_mpagospesos_lag1, Master_mpagospesos) as Master_mpagospesos_delta1,
    Master_mpagospesos - COALESCE(Master_mpagospesos_lag2, Master_mpagospesos) as Master_mpagospesos_delta2,
    Master_mpagospesos - COALESCE(Master_mpagospesos_lag3, Master_mpagospesos) as Master_mpagospesos_delta3,
    Master_mpagosdolares - COALESCE(Master_mpagosdolares_lag1, Master_mpagosdolares) as Master_mpagosdolares_delta1,
    Master_mpagosdolares - COALESCE(Master_mpagosdolares_lag2, Master_mpagosdolares) as Master_mpagosdolares_delta2,
    Master_mpagosdolares - COALESCE(Master_mpagosdolares_lag3, Master_mpagosdolares) as Master_mpagosdolares_delta3,
    Master_mconsumototal - COALESCE(Master_mconsumototal_lag1, Master_mconsumototal) as Master_mconsumototal_delta1,
    Master_mconsumototal - COALESCE(Master_mconsumototal_lag2, Master_mconsumototal) as Master_mconsumototal_delta2,
    Master_mconsumototal - COALESCE(Master_mconsumototal_lag3, Master_mconsumototal) as Master_mconsumototal_delta3,
    Master_mpagominimo - COALESCE(Master_mpagominimo_lag1, Master_mpagominimo) as Master_mpagominimo_delta1,
    Master_mpagominimo - COALESCE(Master_mpagominimo_lag2, Master_mpagominimo) as Master_mpagominimo_delta2,
    Master_mpagominimo - COALESCE(Master_mpagominimo_lag3, Master_mpagominimo) as Master_mpagominimo_delta3,

    ----------------------------------------------------
    -- Deltas para Tarjeta Visa
    ----------------------------------------------------
    Visa_mfinanciacion_limite - COALESCE(Visa_mfinanciacion_limite_lag1, Visa_mfinanciacion_limite) as Visa_mfinanciacion_limite_delta1,
    Visa_mfinanciacion_limite - COALESCE(Visa_mfinanciacion_limite_lag2, Visa_mfinanciacion_limite) as Visa_mfinanciacion_limite_delta2,
    Visa_mfinanciacion_limite - COALESCE(Visa_mfinanciacion_limite_lag3, Visa_mfinanciacion_limite) as Visa_mfinanciacion_limite_delta3,
    Visa_msaldototal - COALESCE(Visa_msaldototal_lag1, Visa_msaldototal) as Visa_msaldototal_delta1,
    Visa_msaldototal - COALESCE(Visa_msaldototal_lag2, Visa_msaldototal) as Visa_msaldototal_delta2,
    Visa_msaldototal - COALESCE(Visa_msaldototal_lag3, Visa_msaldototal) as Visa_msaldototal_delta3,
    Visa_msaldopesos - COALESCE(Visa_msaldopesos_lag1, Visa_msaldopesos) as Visa_msaldopesos_delta1,
    Visa_msaldopesos - COALESCE(Visa_msaldopesos_lag2, Visa_msaldopesos) as Visa_msaldopesos_delta2,
    Visa_msaldopesos - COALESCE(Visa_msaldopesos_lag3, Visa_msaldopesos) as Visa_msaldopesos_delta3,
    Visa_msaldodolares - COALESCE(Visa_msaldodolares_lag1, Visa_msaldodolares) as Visa_msaldodolares_delta1,
    Visa_msaldodolares - COALESCE(Visa_msaldodolares_lag2, Visa_msaldodolares) as Visa_msaldodolares_delta2,
    Visa_msaldodolares - COALESCE(Visa_msaldodolares_lag3, Visa_msaldodolares) as Visa_msaldodolares_delta3,
    Visa_mconsumospesos - COALESCE(Visa_mconsumospesos_lag1, Visa_mconsumospesos) as Visa_mconsumospesos_delta1,
    Visa_mconsumospesos - COALESCE(Visa_mconsumospesos_lag2, Visa_mconsumospesos) as Visa_mconsumospesos_delta2,
    Visa_mconsumospesos - COALESCE(Visa_mconsumospesos_lag3, Visa_mconsumospesos) as Visa_mconsumospesos_delta3,
    Visa_mconsumosdolares - COALESCE(Visa_mconsumosdolares_lag1, Visa_mconsumosdolares) as Visa_mconsumosdolares_delta1,
    Visa_mconsumosdolares - COALESCE(Visa_mconsumosdolares_lag2, Visa_mconsumosdolares) as Visa_mconsumosdolares_delta2,
    Visa_mconsumosdolares - COALESCE(Visa_mconsumosdolares_lag3, Visa_mconsumosdolares) as Visa_mconsumosdolares_delta3,
    Visa_mlimitecompra - COALESCE(Visa_mlimitecompra_lag1, Visa_mlimitecompra) as Visa_mlimitecompra_delta1,
    Visa_mlimitecompra - COALESCE(Visa_mlimitecompra_lag2, Visa_mlimitecompra) as Visa_mlimitecompra_delta2,
    Visa_mlimitecompra - COALESCE(Visa_mlimitecompra_lag3, Visa_mlimitecompra) as Visa_mlimitecompra_delta3,
    Visa_madelantopesos - COALESCE(Visa_madelantopesos_lag1, Visa_madelantopesos) as Visa_madelantopesos_delta1,
    Visa_madelantopesos - COALESCE(Visa_madelantopesos_lag2, Visa_madelantopesos) as Visa_madelantopesos_delta2,
    Visa_madelantopesos - COALESCE(Visa_madelantopesos_lag3, Visa_madelantopesos) as Visa_madelantopesos_delta3,
    Visa_madelantodolares - COALESCE(Visa_madelantodolares_lag1, Visa_madelantodolares) as Visa_madelantodolares_delta1,
    Visa_madelantodolares - COALESCE(Visa_madelantodolares_lag2, Visa_madelantodolares) as Visa_madelantodolares_delta2,
    Visa_madelantodolares - COALESCE(Visa_madelantodolares_lag3, Visa_madelantodolares) as Visa_madelantodolares_delta3,
    Visa_mpagado - COALESCE(Visa_mpagado_lag1, Visa_mpagado) as Visa_mpagado_delta1,
    Visa_mpagado - COALESCE(Visa_mpagado_lag2, Visa_mpagado) as Visa_mpagado_delta2,
    Visa_mpagado - COALESCE(Visa_mpagado_lag3, Visa_mpagado) as Visa_mpagado_delta3,
    Visa_mpagospesos - COALESCE(Visa_mpagospesos_lag1, Visa_mpagospesos) as Visa_mpagospesos_delta1,
    Visa_mpagospesos - COALESCE(Visa_mpagospesos_lag2, Visa_mpagospesos) as Visa_mpagospesos_delta2,
    Visa_mpagospesos - COALESCE(Visa_mpagospesos_lag3, Visa_mpagospesos) as Visa_mpagospesos_delta3,
    Visa_mpagosdolares - COALESCE(Visa_mpagosdolares_lag1, Visa_mpagosdolares) as Visa_mpagosdolares_delta1,
    Visa_mpagosdolares - COALESCE(Visa_mpagosdolares_lag2, Visa_mpagosdolares) as Visa_mpagosdolares_delta2,
    Visa_mpagosdolares - COALESCE(Visa_mpagosdolares_lag3, Visa_mpagosdolares) as Visa_mpagosdolares_delta3,
    Visa_mconsumototal - COALESCE(Visa_mconsumototal_lag1, Visa_mconsumototal) as Visa_mconsumototal_delta1,
    Visa_mconsumototal - COALESCE(Visa_mconsumototal_lag2, Visa_mconsumototal) as Visa_mconsumototal_delta2,
    Visa_mconsumototal - COALESCE(Visa_mconsumototal_lag3, Visa_mconsumototal) as Visa_mconsumototal_delta3,
    Visa_mpagominimo - COALESCE(Visa_mpagominimo_lag1, Visa_mpagominimo) as Visa_mpagominimo_delta1,
    Visa_mpagominimo - COALESCE(Visa_mpagominimo_lag2, Visa_mpagominimo) as Visa_mpagominimo_delta2,
    Visa_mpagominimo - COALESCE(Visa_mpagominimo_lag3, Visa_mpagominimo) as Visa_mpagominimo_delta3
FROM
    competencia_03_con_lags_masivos;

,Success


In [9]:
%%sql
CREATE OR REPLACE TABLE competencia_04_con_ranks_masivos AS
SELECT
    *,
    -- Rankings de Rentabilidad y Márgenes
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mrentabilidad) as mrentabilidad_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mrentabilidad_annual) as mrentabilidad_annual_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcomisiones) as mcomisiones_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mactivos_margen) as mactivos_margen_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mpasivos_margen) as mpasivos_margen_rank,

    -- Rankings de Saldos en Cuentas
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcuenta_corriente_adicional) as mcuenta_corriente_adicional_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcuenta_corriente) as mcuenta_corriente_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcaja_ahorro) as mcaja_ahorro_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcaja_ahorro_adicional) as mcaja_ahorro_adicional_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcaja_ahorro_dolares) as mcaja_ahorro_dolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcuentas_saldo) as mcuentas_saldo_rank,

    -- Rankings de Consumos y Transacciones
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mautoservicio) as mautoservicio_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_visa_consumo) as mtarjeta_visa_consumo_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_master_consumo) as mtarjeta_master_consumo_rank,

    -- Rankings de Deudas por Préstamos
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mprestamos_personales) as mprestamos_personales_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mprestamos_prendarios) as mprestamos_prendarios_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mprestamos_hipotecarios) as mprestamos_hipotecarios_rank,

    -- Rankings de Inversiones
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mplazo_fijo_dolares) as mplazo_fijo_dolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mplazo_fijo_pesos) as mplazo_fijo_pesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY minversion1_pesos) as minversion1_pesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY minversion1_dolares) as minversion1_dolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY minversion2) as minversion2_rank,

    -- Rankings de Ingresos y Pagos
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mpayroll) as mpayroll_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mpayroll2) as mpayroll2_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcuenta_debitos_automaticos) as mcuenta_debitos_automaticos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mttarjeta_visa_debitos_automaticos) as mttarjeta_visa_debitos_automaticos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mttarjeta_master_debitos_automaticos) as mttarjeta_master_debitos_automaticos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mpagodeservicios) as mpagodeservicios_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mpagomiscuentas) as mpagomiscuentas_rank,

    -- Rankings de Descuentos y Comisiones
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcajeros_propios_descuentos) as mcajeros_propios_descuentos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_visa_descuentos) as mtarjeta_visa_descuentos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_master_descuentos) as mtarjeta_master_descuentos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcomisiones_mantenimiento) as mcomisiones_mantenimiento_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcomisiones_otras) as mcomisiones_otras_rank,

    -- Rankings de Forex y Transferencias
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mforex_buy) as mforex_buy_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mforex_sell) as mforex_sell_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mtransferencias_recibidas) as mtransferencias_recibidas_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mtransferencias_emitidas) as mtransferencias_emitidas_rank,

    -- Rankings de Cheques y ATM
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mextraccion_autoservicio) as mextraccion_autoservicio_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcheques_depositados) as mcheques_depositados_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcheques_emitidos) as mcheques_emitidos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcheques_depositados_rechazados) as mcheques_depositados_rechazados_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY mcheques_emitidos_rechazados) as mcheques_emitidos_rechazados_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY matm) as matm_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY matm_other) as matm_other_rank,

    -- Rankings de Tarjeta Master
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mfinanciacion_limite) as Master_mfinanciacion_limite_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_msaldototal) as Master_msaldototal_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_msaldopesos) as Master_msaldopesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_msaldodolares) as Master_msaldodolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mconsumospesos) as Master_mconsumospesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mconsumosdolares) as Master_mconsumosdolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mlimitecompra) as Master_mlimitecompra_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_madelantopesos) as Master_madelantopesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_madelantodolares) as Master_madelantodolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mpagado) as Master_mpagado_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mpagospesos) as Master_mpagospesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mpagosdolares) as Master_mpagosdolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mconsumototal) as Master_mconsumototal_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Master_mpagominimo) as Master_mpagominimo_rank,

    -- Rankings de Tarjeta Visa
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mfinanciacion_limite) as Visa_mfinanciacion_limite_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_msaldototal) as Visa_msaldototal_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_msaldopesos) as Visa_msaldopesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_msaldodolares) as Visa_msaldodolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mconsumospesos) as Visa_mconsumospesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mconsumosdolares) as Visa_mconsumosdolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mlimitecompra) as Visa_mlimitecompra_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_madelantopesos) as Visa_madelantopesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_madelantodolares) as Visa_madelantodolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mpagado) as Visa_mpagado_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mpagospesos) as Visa_mpagospesos_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mpagosdolares) as Visa_mpagosdolares_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mconsumototal) as Visa_mconsumototal_rank,
    NTILE(100) OVER (PARTITION BY foto_mes ORDER BY Visa_mpagominimo) as Visa_mpagominimo_rank
FROM
    competencia_04_con_deltas; -- Usamos la tabla que ya tiene los deltas y lags

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [10]:
%%sql
COPY competencia_04_con_ranks_masivos TO '/home/augusto/Desktop/dmeyf2025/Data/competencia_miranda.csv' (FORMAT CSV, HEADER TRUE);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [11]:
%%sql
select
    *
from competencia_04_con_ranks_masivos

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mconsumospesos_rank,Visa_mconsumosdolares_rank,Visa_mlimitecompra_rank,Visa_madelantopesos_rank,Visa_madelantodolares_rank,Visa_mpagado_rank,Visa_mpagospesos_rank,Visa_mpagosdolares_rank,Visa_mconsumototal_rank,Visa_mpagominimo_rank
0,565006382,202106,1,0,0,42,215,-4347.12,-27997.77,2628.45,...,64,63,30,51,36,79,19,1,64,90
1,465296488,202106,1,0,0,50,248,2070.80,10451.74,4801.13,...,85,61,87,31,12,17,1,32,85,90
2,616096964,202106,1,0,0,35,196,4515.67,-8929.53,9074.26,...,78,66,77,61,31,95,5,4,78,90
3,644704502,202106,1,0,0,54,171,8612.76,79447.19,7240.38,...,13,29,10,48,50,67,24,17,13,90
4,804204671,202106,1,0,0,40,159,-13910.47,-88145.54,4374.76,...,57,69,76,46,32,20,8,19,57,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978434,328581867,202105,1,0,0,65,265,928.99,7743.64,611.91,...,41,8,88,35,9,79,62,70,41,27
978435,331304386,202105,1,0,0,65,159,894.88,11407.29,1192.64,...,41,8,42,16,15,4,65,51,41,27
978436,614850718,202105,1,0,0,49,195,289.01,16752.53,258.27,...,41,9,7,84,73,38,47,73,41,27
978437,890971103,202105,1,0,0,37,131,810.21,7932.06,111.25,...,41,9,65,22,5,41,63,63,41,27


In [12]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01 limit 10

,Master_Fvencimiento,Visa_Fvencimiento,tc_fvencimiento_mayor,tc_fvencimiento_menor
0,-484,-849,-484,-849
1,-514,-1063,-514,-1063
2,-1610,-1702,-1610,-1702
3,-1307,-879,-879,-1307
4,-971,-1337,-971,-1337
5,-545,-606,-545,-606
6,-302,-1763,-302,-1763
7,-788,-1184,-788,-1184
8,-545,-1094,-545,-1094
9,<NA>,-149,-149,-149


Lo siguiente es querer operar dos variables, como por ejemplo sumarla. Esto es sencillo


In [13]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,42243.57,43251.02,85494.59
1,0.00,11310.74,11310.74
2,0.00,19029.85,19029.85
3,0.00,20282.89,20282.89
4,2477.64,32883.64,35361.28
5,0.00,210616.85,210616.85
6,21720.62,19216.74,40937.36
7,13535.96,85963.02,99498.98
8,0.00,118238.88,118238.88
9,NaN,29913.83,NaN


Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null.

In [14]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,29913.83,NaN
1,NaN,5591.57,NaN
2,NaN,NaN,NaN
3,NaN,7485.11,NaN
4,NaN,3580.57,NaN
5,NaN,135524.48,NaN
6,NaN,3667.68,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,26113.14,NaN


Esto no siempre es deseable y puede ser fácilmente evitable

In [15]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01 limit 10

,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,42243.57,43251.02,85494.59
1,0.00,11310.74,11310.74
2,0.00,19029.85,19029.85
3,0.00,20282.89,20282.89
4,2477.64,32883.64,35361.28
5,0.00,210616.85,210616.85
6,21720.62,19216.74,40937.36
7,13535.96,85963.02,99498.98
8,0.00,118238.88,118238.88
9,NaN,29913.83,29913.83


In [16]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

,Success


In [17]:
%%sql
select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10


,Master_msaldototal,Visa_msaldototal,tc_saldo_total
0,NaN,585.08,585.08
1,NaN,7677.36,7677.36
2,NaN,34133.15,34133.15
3,NaN,10260.29,10260.29
4,NaN,12022.02,12022.02
5,NaN,20447.25,20447.25
6,NaN,4110.37,4110.37
7,NaN,26359.46,26359.46
8,NaN,38355.42,38355.42
9,NaN,588.49,588.49


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [18]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


,foto_mes,numero_de_cliente,cliente_antiguedad,cliente_antiguedad_2,cliente_antiguedad_3,cliente_antiguedad_4,cliente_antiguedad_5,cliente_antiguedad_6
0,202101,249221323,93,1,0.379128,0.382611,2,4
1,202102,249221323,94,2,0.381582,0.385027,2,4
2,202103,249221323,95,3,0.384825,0.388262,2,4
3,202104,249221323,96,4,0.387151,0.390593,2,4
4,202105,249221323,97,5,0.389944,0.393394,2,4
...,...,...,...,...,...,...,...,...
978434,202106,1598368433,1,1,0.000000,0.001808,1,1
978435,202106,1598419415,1,1,0.000000,0.001808,1,1
978436,202106,1598425905,1,1,0.000000,0.001808,1,1
978437,202106,1598470389,1,1,0.000000,0.001808,1,1


Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [19]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,249320580,202101,61,<NA>
1,249320580,202102,64,61
2,249320580,202103,82,64
3,249320580,202104,82,82
4,249320580,202105,83,82
5,249320580,202106,67,83
6,249368642,202101,75,<NA>
7,249368642,202102,71,75
8,249368642,202103,77,71
9,249368642,202104,74,77


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [20]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,770475538,202105,85,85,0,-14
1,770475538,202106,92,85,7,7
2,771081347,202101,107,<NA>,<NA>,<NA>
3,771081347,202102,95,107,-12,<NA>
4,771081347,202103,90,95,-5,-17
5,771081347,202104,95,90,5,0
6,771081347,202105,107,95,12,17
7,771081347,202106,100,107,-7,5
8,771291519,202101,162,<NA>,<NA>,<NA>
9,771291519,202102,155,162,-7,<NA>


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [21]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,lag_2_ctrx_quarter,lag_3_ctrx_quarter,avg_3_ctrx_quarter
0,249221323,202106,91,81,89,89,87.500000
1,249221323,202105,81,89,89,97,89.000000
2,249221323,202104,89,89,97,82,89.250000
3,249221323,202103,89,97,82,<NA>,89.333333
4,249221323,202102,97,82,<NA>,<NA>,89.500000
5,249221323,202101,82,<NA>,<NA>,<NA>,82.000000
6,249227600,202106,150,170,155,147,155.500000
7,249227600,202105,170,155,147,126,149.500000
8,249227600,202104,155,147,126,148,144.000000
9,249227600,202103,147,126,148,<NA>,140.333333


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [22]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_3,ctrx_quarter_max_3,ctrx_quarter_min_3
0,249244449,202101,65,65.000000,65,65
1,249244449,202102,78,71.500000,78,65
2,249244449,202103,79,74.000000,79,65
3,249244449,202104,85,76.750000,85,65
4,249244449,202105,82,81.000000,85,78
5,249244449,202106,82,82.000000,85,79
6,249255456,202101,98,98.000000,98,98
7,249255456,202102,105,101.500000,105,98
8,249255456,202103,99,100.666667,105,98
9,249255456,202104,103,101.250000,105,98


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [23]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_slope_3
0,599440939,202105,161,-3.8
1,599440939,202106,166,1.2
2,599453116,202101,104,NaN
3,599453116,202102,108,4.0
4,599453116,202103,73,-15.5
5,599453116,202104,68,-14.3
6,599453116,202105,111,0.4
7,599453116,202106,246,56.2
8,599488266,202101,174,NaN
9,599488266,202102,147,-27.0


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [24]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']


Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [25]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)



, regr_slope(active_quarter, cliente_antiguedad) over ventana_3 as ctrx_active_quarter_slope_3
, regr_slope(cliente_vip, cliente_antiguedad) over ventana_3 as ctrx_cliente_vip_slope_3
, regr_slope(internet, cliente_antiguedad) over ventana_3 as ctrx_internet_slope_3
, regr_slope(cliente_edad, cliente_antiguedad) over ventana_3 as ctrx_cliente_edad_slope_3
, regr_slope(cliente_antiguedad, cliente_antiguedad) over ventana_3 as ctrx_cliente_antiguedad_slope_3
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_3 as ctrx_mrentabilidad_slope_3





Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [26]:
%%sql
COPY competencia_01 TO '{dataset_path}competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE);


,Success
